# PyLASCA
a tool for ...

## square detector
detect landmarks on a real color image for future coordinate transformation 

by

Sebastian Malkusch

2020-04-15

Institute for clinical pharmacology,

data science division,

Goethe-University Frankfurt,

Germany

malkusch@med.uni-frankfurt.de

www.smb.uni-frankfurt.de

http://share.smb.uni-frankfurt.de

## Load libraries

In [1]:
import cv2 as cv
import numpy as np
from ipywidgets import interact
from pyLASCA.gui import squareWidgets
from pyLASCA.source import fileNames
from pyLASCA.source import squareDetector
from pyLASCA.source import sigmaDetector
widgets = squareWidgets.SquareWidgets()
fn = fileNames.FileNames()
vertex = squareDetector.SquareDetector()
sigma = sigmaDetector.SigmaDetector()

PyLascaWidgets initialized


## Import
Load an rgb real color image on which landmarks are to be detected.

In [2]:
display(widgets.pathText, widgets.pathButton)

Text(value='', description='path to file', placeholder='enter a string', style=DescriptionStyle(description_wi…

Button(description='browse', icon='check', style=ButtonStyle(), tooltip='Click me')

In [3]:
display(widgets.importButton)

def runImport(event):
    widgets.clearOutput()
    display(widgets.importButton)
    if(widgets.fileName):
        fn.fileName = widgets.fileName
        fn.splitFileName()
        vertex.loadImage(fn.fileName)
        sigma.updateSeed()
        sigma.loadImage(fn.fileName)
        vertex.drawImage()
        print(fn.fileName)
    
widgets.importButton.on_click(runImport)

Button(description='import', icon='check', style=ButtonStyle(), tooltip='Click me')

## Sigma Detection

In [4]:
display(widgets.sigmaAnalysisButton)

def runSigmaAnalysis(event):
    widgets.clearOutput()
    display(widgets.sigmaAnalysisButton)
    if widgets.fileName:
        sigma.gui()
        widgets.sigmaText.value = sigma.sigma
        print("User defined sigme: %.1f" %(sigma.sigma))
        
        

widgets.sigmaAnalysisButton.on_click(runSigmaAnalysis)

Button(description='analysis', icon='check', style=ButtonStyle(), tooltip='Click me')

## Vertex Analysis

In [5]:
display(widgets.pxlSizeText, widgets.sigmaText, widgets.sigmaSlider, widgets.vertrexAnalysisButton)

def runAnalysis(event):
    widgets.clearOutput()
    display(widgets.pxlSizeText, widgets.sigmaText, widgets.sigmaSlider, widgets.vertrexAnalysisButton)
    if widgets.fileName:
        
        vertex.pxlSize = widgets.pxlSizeText.value
        vertex.sigma = widgets.sigmaText.value
        vertex.analyzeImage()
        vertex.detectVertexes()
        widgets.vertexSelector.options = vertex.vertexArray()
        widgets.vertexSelector.value = -1
        vertex.drawLines()

widgets.vertrexAnalysisButton.on_click(runAnalysis)

BoundedFloatText(value=120.0, description='pxl size [mm]', max=200.0, step=0.1, style=DescriptionStyle(descrip…

BoundedFloatText(value=5.0, description='sigma', max=20.0, min=0.1, step=0.1, style=DescriptionStyle(descripti…

FloatSlider(value=5.0, continuous_update=False, description='sigma', max=20.0, min=0.1, readout_format='.3f', …

Button(description='analysis', icon='check', style=ButtonStyle(), tooltip='Click me')

## Refinement

In [6]:
def updateImage(x):
    if widgets.fileName:
        vertex.drawVertexes(index = x)
        

interact(updateImage, x=widgets.vertexSelector);

interactive(children=(Select(description='Select', options=(-1,), rows=1, value=-1), Output()), _dom_classes=(…

In [7]:
display(widgets.deleteButton)

def deleteLine(event):
    widgets.clearOutput()
    display(widgets.deleteButton)
    if (widgets.vertexSelector.value != -1):
        vertex.deleteVertex(index = widgets.vertexSelector.value)
        widgets.vertexSelector.options = vertex.vertexArray()
        widgets.vertexSelector.value = -1
    if(np.shape(vertex.vertexes)[0] == 4):
        vertex.detectLandmarks()
        display(vertex.landmarks)

widgets.deleteButton.on_click(deleteLine)

Button(description='delete', icon='check', style=ButtonStyle(), tooltip='Click me')

## Save results

In [8]:
display(widgets.saveButton)

def saveLandmarks(event):
    widgets.clearOutput()
    display(widgets.saveButton)
    if (len(vertex.vertexes) == 4):
        fn.updateDateString()
        fn.supplementalInformation = str("Landmarks_sig_%.1f" %(vertex.sigma))
        fn.suffix = "csv"
        fn.mergeFileName()
        vertex.saveLandmarks(fileName = fn.outFileName)
    else:
        print("Wrong number of landmarks. Was %i, need to be 4." %(len(vertex.vertexes)))

widgets.saveButton.on_click(saveLandmarks)

Button(description='save', icon='check', style=ButtonStyle(), tooltip='Click me')